In [5]:
import os
import numpy as np

ddict = np.load("financial/20210104/159920.SZ.Compressed.npz")
for key, value in enumerate(ddict):
    print(key, value)
    # print(key, value, ddict[value])
    temp = ddict[value]

path1 = "financial/"  # 输入一级文件夹地址
files1 = os.listdir(path1)  # 读入一级文件夹
num1 = len(files1)
num2 = []  # 建立空列表
for i in range(num1):
    path2 = path1 + '//' + files1[i]
    files2 = os.listdir(path2)
    num2.append(len(files2))

print("所有二级文件夹名:")
print(files1)  # 打印二级文件夹名称
print("所有二级文件夹中的文件个数:")
print(num2)  # 打印二级文件夹中的文件个数

print("对应输出:")
xinhua = dict(zip(files1, num2))  # 将二级文件夹名称和所含文件个数组合成字典
for key, value in xinhua.items():  # 将二级文件夹名称和所含文件个数对应输出
    print('{key}:{value}'.format(key=key, value=value))

# 20210514/512690  20210326/512480



0 code
1 date
2 name
3 openPrice
4 preClosePrice
5 timestamp
6 totalVolume
7 totalAskVolume
8 totalBidVolume
9 totalMoney
10 lastPrice
11 tradeCount
12 askAvgPrice
13 bidAvgPrice
14 askPrice
15 askVolume
16 bidPrice
17 bidVolume
所有二级文件夹名:
['20210604', '20220401', '20210414', '20210805', '20220527', '20210602', '20210310', '20210223', '20210806', '20210511', '20210422', '20210312', '20210304', '20210830', '20211025', '20211130', '20210823', '20220311', '20210202', '20220429', '20210728', '20210616', '20220321', '20220113', '20210713', '20211115', '20210611', '20210513', '20220126', '20210706', '20210210', '20210811', '20211015', '20220510', '20220207', '20220216', '20220308', '20210518', '20210429', '20210322', '20210831', '20220211', '20210106', '20220215', '20220214', '20210514', '20210305', '20210108', '20210930', '20220225', '20210114', '20211230', '20220518', '20220330', '20211124', '20210319', '20210112', '20210517', '20210506', '20211104', '20220302', '20210609', '20210402', '202

In [6]:
import numpy as np
import os

index1 = ["159920", "159928", "159949", "159995"]
index2 = ["510500","512400","512880","515700","510880","512480","513050","515790","510900","512660","513090","518880","512000","512690","515030","588000", "510050","512010","512760","515050","510300","512170","512800","515220"]
outputdir = "data/"
path1 = "financial/"
k = 48 #window_size
threshold = 0.0003  #threshold
ignoreIndex = 200 #Ignore Index

files1 = os.listdir(path1)
num1 = len(files1)
files1.sort()

In [7]:
def checkname(id):
    if id in index1:
        filename = id + ".SZ.Compressed.npz"
    elif id in index2:
        filename = id + ".SH.Compressed.npz"
    return filename

def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

def check(target, ignoreIndex=100):
    newIndex = np.all(target!=0, axis=1)
    newIndex[ignoreIndex:-ignoreIndex] = True
    output = target[newIndex]
#     output = numba_loops_fill(output)
    return output

def numba_loops_fill(arr):
    '''Numba decorator solution provided by shx2.'''
    out = arr.copy()
    for row_idx in range(out.shape[0]):
        for col_idx in range(1, out.shape[1]):
            if out[row_idx, col_idx] == 0:
                out[row_idx, col_idx] = out[row_idx, col_idx - 1]
    return out

In [10]:
def preprocess(id):
    dat = None
    # id = index1[0]
    for i in range(num1):
        path2 = path1 + files1[i]
        files2 = os.listdir(path2)
        for elem in files2:
            if id in elem:
                print(path2 + "/" + checkname(id))
                ddict = np.load(path2 + "/" + elem)
                askPrice, askVolume, bidPrice, bidVolume = \
                    ddict["askPrice"][:,:5], ddict['askVolume'][:,:5], ddict['bidPrice'][:,:5], ddict['bidVolume'][:,:5]
                # print(askPrice.shape, askVolume.shape, bidPrice.shape, bidVolume.shape)
                date = ddict["timestamp"]
                askPriceNew = check(askPrice)
                # askPriceNew = np.delete(askPrice, np.where((askPrice == 0))[0], axis=0)
                askVolumeNew = check(askVolume)
                bidPriceNew = check(bidPrice)
                bidVolumeNew =  check(bidVolume)

                # AskPriceNew = askPrice[100:-100]
                # AskVolumeNew = askVolume[100:-100]
                # BidPriceNew = bidPrice[100:-100]
                # BidVolumeNew = bidVolume[100:-100]

                pa = askPriceNew.min(axis=1)
                pb = bidPriceNew.max(axis=1)
                pts = (pa + pb) / 2

                pm = moving_average(pts, k)
                for i in range(k-1):
                    pm = np.append(pm, pm[-1])
                perc = pm / pts - 1
                label = [1 if x > threshold else -1 if x < -threshold else 0 for x in perc]
                label = np.array(label)[..., np.newaxis]

                temp = askPriceNew
                temp = np.append(temp, askVolumeNew, axis=1)
                temp = np.append(temp, bidPriceNew, axis=1)
                temp = np.append(temp, bidVolumeNew, axis=1)
                temp = temp / 10000.0
                if 0 in temp:
                    print("No!!!")

                temp = np.append(temp, label, axis=1)
                print(temp.shape)
                dat = temp if dat is None else np.append(dat, temp, axis=0)
                print(dat.shape)

    np.savez(outputdir + str(id) + "_" + str(k) +".npz", dat)
    print(id, dat.shape)
    print("*" * 100)

In [11]:
preprocess("159920")

financial/20210104/159920.SZ.Compressed.npz
(4626, 21)
(4626, 21)
financial/20210105/159920.SZ.Compressed.npz
(4616, 21)
(9242, 21)
financial/20210106/159920.SZ.Compressed.npz
(4619, 21)
(13861, 21)
financial/20210107/159920.SZ.Compressed.npz
(4630, 21)
(18491, 21)
financial/20210108/159920.SZ.Compressed.npz
(4677, 21)
(23168, 21)
financial/20210111/159920.SZ.Compressed.npz
(4738, 21)
(27906, 21)
financial/20210112/159920.SZ.Compressed.npz
(4716, 21)
(32622, 21)
financial/20210113/159920.SZ.Compressed.npz
(4688, 21)
(37310, 21)
financial/20210114/159920.SZ.Compressed.npz
(4702, 21)
(42012, 21)
financial/20210115/159920.SZ.Compressed.npz
(4737, 21)
(46749, 21)
financial/20210118/159920.SZ.Compressed.npz
(4740, 21)
(51489, 21)
financial/20210119/159920.SZ.Compressed.npz
(4743, 21)
(56232, 21)
financial/20210120/159920.SZ.Compressed.npz
(4743, 21)
(60975, 21)
financial/20210121/159920.SZ.Compressed.npz
(4752, 21)
(65727, 21)
financial/20210122/159920.SZ.Compressed.npz
(4743, 21)
(70470, 2

(570592, 21)
financial/20210707/159920.SZ.Compressed.npz
(4730, 21)
(575322, 21)
financial/20210708/159920.SZ.Compressed.npz
(4740, 21)
(580062, 21)
financial/20210709/159920.SZ.Compressed.npz
(4720, 21)
(584782, 21)
financial/20210712/159920.SZ.Compressed.npz
(4629, 21)
(589411, 21)
financial/20210713/159920.SZ.Compressed.npz
(4652, 21)
(594063, 21)
financial/20210714/159920.SZ.Compressed.npz
(4539, 21)
(598602, 21)
financial/20210715/159920.SZ.Compressed.npz
(4607, 21)
(603209, 21)
financial/20210716/159920.SZ.Compressed.npz
(4570, 21)
(607779, 21)
financial/20210719/159920.SZ.Compressed.npz
(4667, 21)
(612446, 21)
financial/20210720/159920.SZ.Compressed.npz
(4718, 21)
(617164, 21)
financial/20210721/159920.SZ.Compressed.npz
(4710, 21)
(621874, 21)
financial/20210722/159920.SZ.Compressed.npz
(4617, 21)
(626491, 21)
financial/20210723/159920.SZ.Compressed.npz
(4657, 21)
(631148, 21)
financial/20210726/159920.SZ.Compressed.npz
(4743, 21)
(635891, 21)
financial/20210727/159920.SZ.Compre

(1131267, 21)
financial/20220104/159920.SZ.Compressed.npz
(4625, 21)
(1135892, 21)
financial/20220105/159920.SZ.Compressed.npz
(4729, 21)
(1140621, 21)
financial/20220106/159920.SZ.Compressed.npz
(4715, 21)
(1145336, 21)
financial/20220107/159920.SZ.Compressed.npz
(4702, 21)
(1150038, 21)
financial/20220110/159920.SZ.Compressed.npz
(4606, 21)
(1154644, 21)
financial/20220111/159920.SZ.Compressed.npz
(4645, 21)
(1159289, 21)
financial/20220112/159920.SZ.Compressed.npz
(4721, 21)
(1164010, 21)
financial/20220113/159920.SZ.Compressed.npz
(4639, 21)
(1168649, 21)
financial/20220114/159920.SZ.Compressed.npz
(4545, 21)
(1173194, 21)
financial/20220117/159920.SZ.Compressed.npz
(4629, 21)
(1177823, 21)
financial/20220118/159920.SZ.Compressed.npz
(4546, 21)
(1182369, 21)
financial/20220119/159920.SZ.Compressed.npz
(4608, 21)
(1186977, 21)
financial/20220120/159920.SZ.Compressed.npz
(4715, 21)
(1191692, 21)
financial/20220121/159920.SZ.Compressed.npz
(4679, 21)
(1196371, 21)
financial/20220124/1

In [7]:
for id in index1:
    preprocess(id)
for id in index2:
    preprocess(id)

financial/20210104/159920.SZ.Compressed.npz
(4626, 41)
(4626, 41)
financial/20210105/159920.SZ.Compressed.npz
(4616, 41)
(9242, 41)
financial/20210106/159920.SZ.Compressed.npz
(4619, 41)
(13861, 41)
financial/20210107/159920.SZ.Compressed.npz
(4630, 41)
(18491, 41)
financial/20210108/159920.SZ.Compressed.npz
(4677, 41)
(23168, 41)
financial/20210111/159920.SZ.Compressed.npz
(4738, 41)
(27906, 41)
financial/20210112/159920.SZ.Compressed.npz
(4716, 41)
(32622, 41)
financial/20210113/159920.SZ.Compressed.npz
(4688, 41)
(37310, 41)
financial/20210114/159920.SZ.Compressed.npz
(4702, 41)
(42012, 41)
financial/20210115/159920.SZ.Compressed.npz
(4737, 41)
(46749, 41)
financial/20210118/159920.SZ.Compressed.npz
(4740, 41)
(51489, 41)
financial/20210119/159920.SZ.Compressed.npz
(4743, 41)
(56232, 41)
financial/20210120/159920.SZ.Compressed.npz
(4743, 41)
(60975, 41)
financial/20210121/159920.SZ.Compressed.npz
(4752, 41)
(65727, 41)
financial/20210122/159920.SZ.Compressed.npz
(4743, 41)
(70470, 4

(4692, 41)
(570592, 41)
financial/20210707/159920.SZ.Compressed.npz
(4730, 41)
(575322, 41)
financial/20210708/159920.SZ.Compressed.npz
(4740, 41)
(580062, 41)
financial/20210709/159920.SZ.Compressed.npz
(4720, 41)
(584782, 41)
financial/20210712/159920.SZ.Compressed.npz
(4629, 41)
(589411, 41)
financial/20210713/159920.SZ.Compressed.npz
(4652, 41)
(594063, 41)
financial/20210714/159920.SZ.Compressed.npz
(4539, 41)
(598602, 41)
financial/20210715/159920.SZ.Compressed.npz
(4607, 41)
(603209, 41)
financial/20210716/159920.SZ.Compressed.npz
(4570, 41)
(607779, 41)
financial/20210719/159920.SZ.Compressed.npz
(4667, 41)
(612446, 41)
financial/20210720/159920.SZ.Compressed.npz
(4718, 41)
(617164, 41)
financial/20210721/159920.SZ.Compressed.npz
(4710, 41)
(621874, 41)
financial/20210722/159920.SZ.Compressed.npz
(4617, 41)
(626491, 41)
financial/20210723/159920.SZ.Compressed.npz
(4657, 41)
(631148, 41)
financial/20210726/159920.SZ.Compressed.npz
(4743, 41)
(635891, 41)
financial/20210727/15992

(4561, 41)
(1131267, 41)
financial/20220104/159920.SZ.Compressed.npz
(4625, 41)
(1135892, 41)
financial/20220105/159920.SZ.Compressed.npz
(4729, 41)
(1140621, 41)
financial/20220106/159920.SZ.Compressed.npz
(4715, 41)
(1145336, 41)
financial/20220107/159920.SZ.Compressed.npz
(4702, 41)
(1150038, 41)
financial/20220110/159920.SZ.Compressed.npz
(4606, 41)
(1154644, 41)
financial/20220111/159920.SZ.Compressed.npz
(4645, 41)
(1159289, 41)
financial/20220112/159920.SZ.Compressed.npz
(4721, 41)
(1164010, 41)
financial/20220113/159920.SZ.Compressed.npz
(4639, 41)
(1168649, 41)
financial/20220114/159920.SZ.Compressed.npz
(4545, 41)
(1173194, 41)
financial/20220117/159920.SZ.Compressed.npz
(4629, 41)
(1177823, 41)
financial/20220118/159920.SZ.Compressed.npz
(4546, 41)
(1182369, 41)
financial/20220119/159920.SZ.Compressed.npz
(4608, 41)
(1186977, 41)
financial/20220120/159920.SZ.Compressed.npz
(4715, 41)
(1191692, 41)
financial/20220121/159920.SZ.Compressed.npz
(4679, 41)
(1196371, 41)
financial

(4713, 41)
(103879, 41)
financial/20210203/159928.SZ.Compressed.npz
(4704, 41)
(108583, 41)
financial/20210204/159928.SZ.Compressed.npz
(4711, 41)
(113294, 41)
financial/20210205/159928.SZ.Compressed.npz
(4740, 41)
(118034, 41)
financial/20210208/159928.SZ.Compressed.npz
(4711, 41)
(122745, 41)
financial/20210209/159928.SZ.Compressed.npz
(4708, 41)
(127453, 41)
financial/20210210/159928.SZ.Compressed.npz
(4724, 41)
(132177, 41)
financial/20210218/159928.SZ.Compressed.npz
(4727, 41)
(136904, 41)
financial/20210219/159928.SZ.Compressed.npz
(4735, 41)
(141639, 41)
financial/20210222/159928.SZ.Compressed.npz
(4741, 41)
(146380, 41)
financial/20210223/159928.SZ.Compressed.npz
(4698, 41)
(151078, 41)
financial/20210224/159928.SZ.Compressed.npz
(4734, 41)
(155812, 41)
financial/20210225/159928.SZ.Compressed.npz
(4703, 41)
(160515, 41)
financial/20210226/159928.SZ.Compressed.npz
(4717, 41)
(165232, 41)
financial/20210301/159928.SZ.Compressed.npz
(4662, 41)
(169894, 41)
financial/20210302/15992

(4642, 41)
(659153, 41)
financial/20210805/159928.SZ.Compressed.npz
(4448, 41)
(663601, 41)
financial/20210806/159928.SZ.Compressed.npz
(4425, 41)
(668026, 41)
financial/20210809/159928.SZ.Compressed.npz
(4677, 41)
(672703, 41)
financial/20210810/159928.SZ.Compressed.npz
(4710, 41)
(677413, 41)
financial/20210811/159928.SZ.Compressed.npz
(4678, 41)
(682091, 41)
financial/20210812/159928.SZ.Compressed.npz
(4688, 41)
(686779, 41)
financial/20210813/159928.SZ.Compressed.npz
(4524, 41)
(691303, 41)
financial/20210816/159928.SZ.Compressed.npz
(4304, 41)
(695607, 41)
financial/20210817/159928.SZ.Compressed.npz
(4660, 41)
(700267, 41)
financial/20210818/159928.SZ.Compressed.npz
(4565, 41)
(704832, 41)
financial/20210819/159928.SZ.Compressed.npz
(4659, 41)
(709491, 41)
financial/20210820/159928.SZ.Compressed.npz
(4740, 41)
(714231, 41)
financial/20210823/159928.SZ.Compressed.npz
(4580, 41)
(718811, 41)
financial/20210824/159928.SZ.Compressed.npz
(4567, 41)
(723378, 41)
financial/20210825/15992

...

(4865, 41)
(1342047, 41)
financial/20220225/512880.SH.Compressed.npz
(4908, 41)
(1346955, 41)
financial/20220228/512880.SH.Compressed.npz
(4855, 41)
(1351810, 41)
financial/20220301/512880.SH.Compressed.npz
(4857, 41)
(1356667, 41)
financial/20220302/512880.SH.Compressed.npz
(4875, 41)
(1361542, 41)
financial/20220303/512880.SH.Compressed.npz
(4867, 41)
(1366409, 41)
financial/20220304/512880.SH.Compressed.npz
(4845, 41)
(1371254, 41)
financial/20220307/512880.SH.Compressed.npz
(4855, 41)
(1376109, 41)
financial/20220308/512880.SH.Compressed.npz
(4857, 41)
(1380966, 41)
financial/20220309/512880.SH.Compressed.npz
(4875, 41)
(1385841, 41)
financial/20220310/512880.SH.Compressed.npz
(4917, 41)
(1390758, 41)
financial/20220311/512880.SH.Compressed.npz
(4891, 41)
(1395649, 41)
financial/20220314/512880.SH.Compressed.npz
(4904, 41)
(1400553, 41)
financial/20220315/512880.SH.Compressed.npz
(4849, 41)
(1405402, 41)
financial/20220316/512880.SH.Compressed.npz
(4889, 41)
(1410291, 41)
financial

(4685, 41)
(270338, 41)
financial/20210330/515700.SH.Compressed.npz
(4681, 41)
(275019, 41)
financial/20210331/515700.SH.Compressed.npz
(4613, 41)
(279632, 41)
financial/20210401/515700.SH.Compressed.npz
(4718, 41)
(284350, 41)
financial/20210402/515700.SH.Compressed.npz
(4723, 41)
(289073, 41)
financial/20210406/515700.SH.Compressed.npz
(4617, 41)
(293690, 41)
financial/20210407/515700.SH.Compressed.npz
(4604, 41)
(298294, 41)
financial/20210408/515700.SH.Compressed.npz
(4660, 41)
(302954, 41)
financial/20210409/515700.SH.Compressed.npz
(4767, 41)
(307721, 41)
financial/20210412/515700.SH.Compressed.npz
(4816, 41)
(312537, 41)
financial/20210413/515700.SH.Compressed.npz
(4626, 41)
(317163, 41)
financial/20210414/515700.SH.Compressed.npz
(4811, 41)
(321974, 41)
financial/20210415/515700.SH.Compressed.npz
(4670, 41)
(326644, 41)
financial/20210416/515700.SH.Compressed.npz
(4630, 41)
(331274, 41)
financial/20210419/515700.SH.Compressed.npz
(4825, 41)
(336099, 41)
financial/20210420/51570

(4712, 41)
(851548, 41)
financial/20210924/515700.SH.Compressed.npz
(4770, 41)
(856318, 41)
financial/20210927/515700.SH.Compressed.npz
(4789, 41)
(861107, 41)
financial/20210928/515700.SH.Compressed.npz
(4810, 41)
(865917, 41)
financial/20210929/515700.SH.Compressed.npz
(4818, 41)
(870735, 41)
financial/20210930/515700.SH.Compressed.npz
(4821, 41)
(875556, 41)
financial/20211008/515700.SH.Compressed.npz
(4805, 41)
(880361, 41)
financial/20211011/515700.SH.Compressed.npz
(4686, 41)
(885047, 41)
financial/20211012/515700.SH.Compressed.npz
(4813, 41)
(889860, 41)
financial/20211013/515700.SH.Compressed.npz
(4796, 41)
(894656, 41)
financial/20211014/515700.SH.Compressed.npz
(4741, 41)
(899397, 41)
financial/20211015/515700.SH.Compressed.npz
(4819, 41)
(904216, 41)
financial/20211018/515700.SH.Compressed.npz
(4802, 41)
(909018, 41)
financial/20211019/515700.SH.Compressed.npz
(4798, 41)
(913816, 41)
financial/20211020/515700.SH.Compressed.npz
(4814, 41)
(918630, 41)
financial/20211021/51570

(4795, 41)
(1424281, 41)
financial/20220328/515700.SH.Compressed.npz
(4755, 41)
(1429036, 41)
financial/20220329/515700.SH.Compressed.npz
(4893, 41)
(1433929, 41)
financial/20220330/515700.SH.Compressed.npz
(4813, 41)
(1438742, 41)
financial/20220331/515700.SH.Compressed.npz
(4834, 41)
(1443576, 41)
financial/20220401/515700.SH.Compressed.npz
(4710, 41)
(1448286, 41)
financial/20220406/515700.SH.Compressed.npz
(4707, 41)
(1452993, 41)
financial/20220407/515700.SH.Compressed.npz
(4666, 41)
(1457659, 41)
financial/20220408/515700.SH.Compressed.npz
(4730, 41)
(1462389, 41)
financial/20220411/515700.SH.Compressed.npz
(4811, 41)
(1467200, 41)
financial/20220412/515700.SH.Compressed.npz
(4721, 41)
(1471921, 41)
financial/20220413/515700.SH.Compressed.npz
(4733, 41)
(1476654, 41)
financial/20220414/515700.SH.Compressed.npz
(4748, 41)
(1481402, 41)
financial/20220415/515700.SH.Compressed.npz
(4803, 41)
(1486205, 41)
financial/20220418/515700.SH.Compressed.npz
(4742, 41)
(1490947, 41)
financial

ValueError: operands could not be broadcast together with shapes (4753,) (4809,) 